# 04 — Resultados de Backtesting

Este notebook analiza los resultados del backtesting del sistema de trading:

1. Métricas clave: Profit Factor (PF), Win Rate (WR), Drawdown
2. Curva de equity simulada
3. Distribución de trades
4. Análisis por temporalidad y dirección
5. Comparación con benchmarks

> **Nota**: Los resultados específicos por señal son confidenciales.
> Se presentan métricas agregadas y metodología de evaluación.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

sns.set_theme(style='whitegrid', palette='muted')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['figure.dpi'] = 100

DATA_DIR = Path('../data/sample')

## 1. Métricas clave de evaluación

### Profit Factor (PF)

$$PF = \frac{\text{Beneficio bruto}}{\text{Pérdida bruta}}$$

- PF < 1.0: Sistema perdedor
- PF = 1.0: Breakeven
- PF > 1.2: Mínimo para cubrir costes reales
- PF > 1.5: Sistema rentable

### Win Rate (WR)

$$WR = \frac{\text{Trades ganadores}}{\text{Trades totales}}$$

### Maximum Drawdown (MDD)

Máxima caída desde un pico de equity. Mide el riesgo del peor escenario.

In [ ]:
# Cargar trades reales del paper trading
trades_path = DATA_DIR / 'trades' / 'paper_trades.csv'
trades = pd.read_csv(trades_path)

# Parsear fechas
trades['entry_dt'] = pd.to_datetime(trades['entry_time'])
trades['exit_dt'] = pd.to_datetime(trades['exit_time'])
trades = trades.sort_values('exit_dt').reset_index(drop=True)

print(f'Total trades: {len(trades)}')
print(f'Periodo: {trades["entry_dt"].min().date()} → {trades["exit_dt"].max().date()}')
print(f'Pares: {trades["symbol"].nunique()} ({sorted(trades["symbol"].unique())})')
print()
trades[['symbol', 'direction', 'pnl_pips', 'exit_reason', 'entry_model']].head(10)

In [ ]:
# Calcular métricas globales
wins = trades[trades['pnl_pips'] > 0]
losses = trades[trades['pnl_pips'] <= 0]

total_trades = len(trades)
win_rate = len(wins) / total_trades if total_trades > 0 else 0
gross_profit = wins['pnl_pips'].sum()
gross_loss = abs(losses['pnl_pips'].sum())
profit_factor = gross_profit / gross_loss if gross_loss > 0 else float('inf')
avg_win = wins['pnl_pips'].mean() if len(wins) > 0 else 0
avg_loss = losses['pnl_pips'].mean() if len(losses) > 0 else 0

# Drawdown
cumulative = trades['pnl_pips'].cumsum()
peak = cumulative.cummax()
drawdown = cumulative - peak
max_drawdown = drawdown.min()

print('=== Métricas Globales ===')
print(f'Trades totales:    {total_trades}')
print(f'Win Rate:          {win_rate:.1%}')
print(f'Profit Factor:     {profit_factor:.2f}')
print(f'P&L total (pips):  {trades["pnl_pips"].sum():.1f}')
print(f'Media ganador:     +{avg_win:.1f} pips')
print(f'Media perdedor:    {avg_loss:.1f} pips')
print(f'Max Drawdown:      {max_drawdown:.1f} pips')

## 2. Curva de equity

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 10), gridspec_kw={'height_ratios': [3, 1]})

# Equity curve
ax1 = axes[0]
ax1.plot(trades['exit_dt'], cumulative, color='steelblue', linewidth=1.5)
ax1.fill_between(trades['exit_dt'], cumulative, alpha=0.1, color='steelblue')
ax1.axhline(0, color='gray', linestyle='--', alpha=0.5)
ax1.set_title('Curva de Equity Acumulada (pips)', fontsize=13)
ax1.set_ylabel('P&L acumulado (pips)')

# Drawdown
ax2 = axes[1]
ax2.fill_between(trades['exit_dt'], drawdown, 0, color='crimson', alpha=0.4)
ax2.plot(trades['exit_dt'], drawdown, color='crimson', linewidth=0.8)
ax2.set_title(f'Drawdown (max: {max_drawdown:.1f} pips)', fontsize=12)
ax2.set_ylabel('Drawdown (pips)')

plt.tight_layout()
plt.show()

## 3. Distribución de P&L por trade

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histograma de P&L
colors = ['green' if x > 0 else 'red' for x in trades['pnl_pips']]
axes[0].hist(trades['pnl_pips'], bins=40, color='steelblue', alpha=0.7, edgecolor='white')
axes[0].axvline(0, color='red', linestyle='--', alpha=0.7)
axes[0].axvline(trades['pnl_pips'].mean(), color='green', linestyle='--',
                label=f'Media: {trades["pnl_pips"].mean():.1f} pips')
axes[0].set_xlabel('P&L (pips)', fontsize=11)
axes[0].set_ylabel('Frecuencia', fontsize=11)
axes[0].set_title('Distribución de P&L por Trade', fontsize=12)
axes[0].legend()

# P&L por razón de salida
if 'exit_reason' in trades.columns:
    exit_stats = trades.groupby('exit_reason').agg(
        count=('pnl_pips', 'count'),
        mean_pnl=('pnl_pips', 'mean'),
        total_pnl=('pnl_pips', 'sum')
    ).sort_values('count', ascending=True)

    colors = ['green' if x > 0 else 'red' for x in exit_stats['mean_pnl']]
    exit_stats['mean_pnl'].plot(kind='barh', ax=axes[1], color=colors, alpha=0.7)
    axes[1].set_xlabel('P&L medio (pips)', fontsize=11)
    axes[1].set_title('P&L Medio por Razón de Salida', fontsize=12)
    axes[1].axvline(0, color='gray', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

## 4. Análisis por par y dirección

In [ ]:
# Métricas por par de divisas
def calc_metrics(group):
    w = group[group['pnl_pips'] > 0]
    l = group[group['pnl_pips'] <= 0]
    gp = w['pnl_pips'].sum()
    gl = abs(l['pnl_pips'].sum())
    return pd.Series({
        'Trades': len(group),
        'WR (%)': len(w) / len(group) * 100 if len(group) > 0 else 0,
        'PF': gp / gl if gl > 0 else np.inf,
        'P&L (pips)': group['pnl_pips'].sum(),
        'Media (pips)': group['pnl_pips'].mean()
    })

by_symbol = trades.groupby('symbol').apply(calc_metrics).round(2)
print('=== Métricas por Par ===')
by_symbol

In [ ]:
# Métricas por dirección
by_direction = trades.groupby('direction').apply(calc_metrics).round(2)
print('=== Métricas por Dirección ===')
by_direction

In [ ]:
# Heatmap: P&L por par × dirección
pivot = trades.pivot_table(values='pnl_pips', index='symbol', columns='direction',
                           aggfunc='sum', fill_value=0)

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(pivot, annot=True, fmt='.1f', cmap='RdYlGn', center=0, ax=ax,
            linewidths=0.5, linecolor='white')
ax.set_title('P&L Total (pips) por Par × Dirección', fontsize=13)
plt.tight_layout()
plt.show()

## 5. Análisis temporal: P&L diario

In [ ]:
# P&L diario
trades['exit_date'] = trades['exit_dt'].dt.date
daily_pnl = trades.groupby('exit_date')['pnl_pips'].sum()

fig, ax = plt.subplots(figsize=(14, 5))
colors = ['green' if x > 0 else 'red' for x in daily_pnl]
ax.bar(range(len(daily_pnl)), daily_pnl.values, color=colors, alpha=0.7)
ax.axhline(0, color='gray', linewidth=0.5)
ax.set_title('P&L Diario (pips)', fontsize=13)
ax.set_ylabel('P&L (pips)')
ax.set_xlabel('Día')

# Estadísticas
positive_days = (daily_pnl > 0).sum()
total_days = len(daily_pnl)
ax.text(0.02, 0.95, f'Días positivos: {positive_days}/{total_days} ({positive_days/total_days*100:.0f}%)',
        transform=ax.transAxes, fontsize=10, verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

## 6. Comparación con benchmark

### ¿Es mejor que el azar?

Para validar que el sistema tiene ventaja real, se compara contra un **benchmark aleatorio**:
un sistema que entra aleatoriamente con el mismo SL/TP.

In [ ]:
# Simulación Monte Carlo: PF de sistemas aleatorios
np.random.seed(42)

n_simulations = 5000
n_trades_sim = len(trades)
random_pfs = []

for _ in range(n_simulations):
    # Simular trades aleatorios con WR = 50% y SL/TP simétrico
    random_pnl = np.where(np.random.random(n_trades_sim) > 0.5, 27.5, -32.5)  # incluyendo costes
    gp = random_pnl[random_pnl > 0].sum()
    gl = abs(random_pnl[random_pnl < 0].sum())
    random_pfs.append(gp / gl if gl > 0 else 1.0)

random_pfs = np.array(random_pfs)

fig, ax = plt.subplots(figsize=(12, 5))
ax.hist(random_pfs, bins=50, density=True, color='lightcoral', alpha=0.7,
        edgecolor='white', label=f'Benchmark aleatorio (n={n_simulations})')
ax.axvline(profit_factor, color='green', linewidth=2.5, linestyle='--',
           label=f'Sistema real: PF={profit_factor:.2f}')
ax.axvline(1.0, color='red', linewidth=1, linestyle=':', label='Breakeven')

p_value = (random_pfs >= profit_factor).mean()
ax.set_title(f'Sistema vs Benchmark Aleatorio (p-value = {p_value:.4f})', fontsize=13)
ax.set_xlabel('Profit Factor', fontsize=11)
ax.set_ylabel('Densidad', fontsize=11)
ax.legend()

plt.tight_layout()
plt.show()

print(f'PF del sistema:    {profit_factor:.2f}')
print(f'PF medio aleatorio: {random_pfs.mean():.3f}')
print(f'p-value:           {p_value:.4f}')
print(f'Conclusión: {"El sistema es SIGNIFICATIVAMENTE mejor que el azar" if p_value < 0.05 else "No se puede distinguir del azar"}')

## 7. Conclusiones

### Métricas de evaluación

El backtesting utiliza métricas robustas que incluyen:
- **Costes de transacción realistas** (spread + slippage + comisiones)
- **Separación cronológica estricta** (datos de test nunca vistos durante entrenamiento)
- **Comparación con benchmark aleatorio** (Monte Carlo)
- **Análisis de drawdown** para evaluar riesgo

### Validez estadística

Un Profit Factor > 1 sostenido en datos out-of-sample, con p-value significativo
frente a un benchmark aleatorio, sugiere que el sistema captura patrones reales del mercado.

> **Siguiente**: `05_paper_trading_analysis.ipynb` — Análisis del paper trading en vivo